In [ ]:
path = "periodic_table.csv"
import pandas as pd
#Load the "periodic_table.csv" file and create a dictionary mapping element symbols to their atomic masses.
data = pd.read_csv(path)
atomic_masses = dict(zip(data['Symbol'], data['AtomicMass']))

15.999


In [ ]:
def calculate_molecular_mass(formula):
    #This function calculates the molecular mass of a chemical formula without parentheses. It parses the formula character by character, identifying elements and their counts, and sums their contributions to the total molecular mass.
    #initialize total mass and index
    total_mass = 0.0
    i = 0
    #loops over the chemical formula
    while i < len(formula):
        #parsing of the element symbol
        element = formula[i]
        i += 1
        if i < len(formula) and formula[i].islower():
            element += formula[i]
            i += 1
        #parsing the count of atoms
        count_str = ''
        while i < len(formula) and formula[i].isdigit():
            count_str += formula[i]
            i += 1
        count = int(count_str) if count_str else 1
        #adding the mass contribution of the element to the total mass
        if element in atomic_masses:
            total_mass += atomic_masses[element] * count
        else:
            raise ValueError(f"Element {element} not found in atomic masses.")
    
    return total_mass

In [ ]:
def calculate_molecular_mass_with_parentheses(formula):
    #this function calculates the molecular mass of a chemical formula that may include parentheses. It does this by parsing the formula character by character, handling elements, counts, and nested parentheses. Finally, at the end, it returns the total molecular mass by summing the contributions from each part of the formula.
    total_mass = 0.0
    i = 0
    #while loop through the formula
    while i < len(formula):
        if formula[i] == '(':
            #find matching ')'
            paren_count = 1
            start = i + 1
            i += 1
            #while loop to find the closing parenthesis
            while i < len(formula) and paren_count > 0:
                if formula[i] == '(':
                    paren_count += 1
                elif formula[i] == ')':
                    paren_count -= 1
                i += 1
            inside = formula[start:i-1]
            
            #get multiplier after parenthesis
            number_str = ""
            while i < len(formula) and formula[i].isdigit():
                number_str += formula[i]
                i += 1
            multiplier = int(number_str) if number_str else 1
            
            #add mass of inside parentheses * multiplier
            total_mass += calculate_molecular_mass_with_parentheses(inside) * multiplier
        
        else:
            #reads chemical element symbols, which can be one or two letters long.
            element = formula[i]
            i += 1
            if i < len(formula) and formula[i].islower():
                element += formula[i]
                i += 1
            
            #reads number of atoms that follow an element or a group of atoms in parentheses.
            number_str = ""
            while i < len(formula) and formula[i].isdigit():
                number_str += formula[i]
                i += 1
            count = int(number_str) if number_str else 1
            #finally, it checks if the element is in the atomic_masses dictionary and adds its contribution to the total mass.
            if element in atomic_masses:
                total_mass += atomic_masses[element] * count
            else:
                raise ValueError(f"Unknown element: {element}")
    
    return total_mass


74.094


In [ ]:
def calculate_molecular_mass_complete(formula):
#function to calculate molecular mass, handling parentheses and hydration water. This function first expands any parentheses in the formula, then calculates the mass of the expanded formula. If the formula includes hydration water (indicated by a dot), it separately calculates the mass of the main compound and the hydration water, summing them for the final molecular mass.
    #internal function to expand parentheses
    def expand_parentheses(f):
        result = ""
        i = 0
        #while loop through the formula
        while i < len(f):
            if f[i] == '(':
                paren_count = 1
                start = i + 1
                i += 1
                #while loop to find the closing parenthesis
                while i < len(f) and paren_count > 0:
                    if f[i] == '(':
                        paren_count += 1
                    elif f[i] == ')':
                        paren_count -= 1
                    i += 1
                inside_parentheses = f[start:i-1]
                number_str = ""
                #while loop to get multiplier after parenthesis
                while i < len(f) and f[i].isdigit():
                    number_str += f[i]
                    i += 1
                #multiplier for the group inside parentheses
                multiplier = int(number_str) if number_str else 1
                expanded_inside = expand_parentheses(inside_parentheses)
                #for loop to repeat the expanded inside parentheses according to the multiplier
                for _ in range(multiplier):
                    result += expanded_inside
            else:
                result += f[i]
                i += 1
        #returns the fully expanded formula
        return result
    
    #internal function to calculate mass from expanded formula
    def calculate_from_expanded(expanded_f):
        element_counts = {}
        i = 0
        #while loop through the expanded formula
        while i < len(expanded_f):
            if expanded_f[i].isupper():
                element = expanded_f[i]
                i += 1
                #if the next character is lowercase, it is part of the element symbol
                if i < len(expanded_f) and expanded_f[i].islower():
                    element += expanded_f[i]
                    i += 1
                number_str = ""
                #while loop to read the number of atoms
                while i < len(expanded_f) and expanded_f[i].isdigit():
                    number_str += expanded_f[i]
                    i += 1
                #counter for the number of atoms
                count = int(number_str) if number_str else 1
                if element in element_counts:
                    element_counts[element] += count
                else:
                    element_counts[element] = count
            else:
                i += 1
        
        total_mass = 0.0
        #for loop to calculate total mass from element counts
        for element, count in element_counts.items():
            if element in atomic_masses:
                total_mass += atomic_masses[element] * count
            else:
                raise ValueError(f"Unknown element: {element}")
        return total_mass
    #check for hydration water indicated by a dot (.)
    if '.' in formula:
        parts = formula.split('.')
        main_compound = parts[0]
        hydration_part = parts[1]
        
        #calculate main compound mass
        expanded_main = expand_parentheses(main_compound)
        main_mass = calculate_from_expanded(expanded_main)
        
        #calculate hydration water mass
        #extract number of water molecules
        i = 0
        number_str = ""
        #while loop to read the number of water molecules
        while i < len(hydration_part) and hydration_part[i].isdigit():
            number_str += hydration_part[i]
            i += 1
        
        water_count = int(number_str) if number_str else 1
        water_formula = hydration_part[i:]  # Should be H2O
        
        #calculate one water molecule mass
        expanded_water = expand_parentheses(water_formula)
        single_water_mass = calculate_from_expanded(expanded_water)
        total_hydration_mass = single_water_mass * water_count
        
        #total mass
        total_mass = main_mass + total_hydration_mass
        
    else:
        #no hydration water, regular calculation
        expanded = expand_parentheses(formula)
        if expanded != formula:
            print(f"Expanded to: {expanded}")
        total_mass = calculate_from_expanded(expanded)
    
    return total_mass


Expanded to: CaOHOH
74.09400000000001
249.69100000000003
